## 获取events-discipline-sports的关系，以及各个国家的获奖总数

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')
%cd dataset
!ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/dataset
classified_country_medal_stats_after1950.csv	    olympic_medals_2000-2020.csv
classified_country_medal_stats_after1984.csv	    olympic_medals_with_noc.csv
classified_country_medal_stats_after1984_useCV.csv  olympic_score_model.pkl
classified_country_medal_stats.csv		    potential_stars.csv
classified_country_medal_stats_cv.csv		    potential_stars_filtered.csv
country.csv					    residual_plot.png
country_medal_probability.csv			    robust_scaler.pkl
country_medal_stats.csv				    scatter_matrix.png
country_olympic_potential.csv			    score_validation.png
data_dictionary.csv				    summerOly_athletes.csv
final_medal_table.csv				    summerOly_hosts.csv
medal_distribution.png				    summerOly_medal_counts.csv
merged_country_data.csv				    summerOly_programs.csv
noc_codes.json


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [4]:
athletes = pd.read_csv('summerOly_athletes.csv')
hosts = pd.read_csv('summerOly_hosts.csv')
medal_counts = pd.read_csv('summerOly_medal_counts.csv')
programs = pd.read_csv('summerOly_programs.csv',encoding='latin-1')
countrys = pd.read_csv('country.csv')

In [5]:
athletes.info()
athletes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252565 entries, 0 to 252564
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Name    252565 non-null  object
 1   Sex     252565 non-null  object
 2   Team    252565 non-null  object
 3   NOC     252565 non-null  object
 4   Year    252565 non-null  int64 
 5   City    252565 non-null  object
 6   Sport   252565 non-null  object
 7   Event   252565 non-null  object
 8   Medal   252565 non-null  object
dtypes: int64(1), object(8)
memory usage: 17.3+ MB


,Name,Sex,Team,NOC,Year,City,Sport,Event,Medal
0,A Dijiang,M,China,CHN,1992,Barcelona,Basketball,Basketball Men's Basketball,No medal
1,A Lamusi,M,China,CHN,2012,London,Judo,Judo Men's Extra-Lightweight,No medal
2,Gunnar Aaby,M,Denmark,DEN,1920,Antwerpen,Football,Football Men's Football,No medal
3,Edgar Aabye,M,Denmark/Sweden,DEN,1900,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Cornelia (-strannood),F,Netherlands,NED,1932,Los Angeles,Athletics,Athletics Women's 100 metres,No medal
...,...,...,...,...,...,...,...,...,...
252560,Sefora Ada,F,Equatorial Guinea,GEQ,2024,Paris,Athletics,Women's 100m,No medal
252561,Emanuela Liuzzi,F,Italy,ITA,2024,Paris,Wrestling,Women's Freestyle 50kg,No medal
252562,Isayah Boers,M,Netherlands,NED,2024,Paris,Athletics,4 x 400m Relay Mixed,Gold
252563,Kevin Staut,M,France,FRA,2024,Paris,Equestrian,Jumping Team,Bronze


In [10]:
# 假设 DataFrame 存储在变量 df 中
df = athletes
# 按 Sport 分组并提取唯一的 Event 列表
sport_events = df.groupby('Sport')['Event'].unique().apply(list).to_dict()

# 保存为 JSON 文件
with open('sport_events.json', 'w', encoding='utf-8') as f:
    json.dump(sport_events, f, ensure_ascii=False, indent=2)

print("保存成功！文件名为 'sport_events.json'")

保存成功！文件名为 'sport_events.json'


In [8]:
programs.info()
programs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 35 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Sport                  74 non-null     object 
 1   Discipline             72 non-null     object 
 2   Code                   74 non-null     object 
 3   Sports Governing Body  74 non-null     object 
 4   1896                   74 non-null     object 
 5   1900                   74 non-null     object 
 6   1904                   74 non-null     object 
 7   1906*                  74 non-null     int64  
 8   1908                   74 non-null     int64  
 9   1912                   74 non-null     object 
 10  1920                   74 non-null     int64  
 11  1924                   74 non-null     object 
 12  1928                   72 non-null     object 
 13  1932                   72 non-null     object 
 14  1936                   72 non-null     object 
 15  1948    

,Sport,Discipline,Code,Sports Governing Body,1896,1900,1904,1906*,1908,1912,...,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,Aquatics,Artistic Swimming,SWA,World Aquatics,0,0,0,0,0,0,...,2,2,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,Aquatics,Diving,DIV,World Aquatics,0,0,2,1,2,4,...,4,4,4.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0
2,Aquatics,Marathon Swimming,OWS,World Aquatics,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
3,Aquatics,Swimming,SWM,World Aquatics,4,7,9,4,6,9,...,31,31,32.0,32.0,32.0,32.0,32.0,32.0,35.0,35.0
4,Aquatics,Water Polo,WPO,World Aquatics,0,1,1,0,1,1,...,1,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Skating,Figure,FSK,ISU,0,0,0,0,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,Ice Hockey,Ice Hockey,IHO,IIHF,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,Total events,Total events,Total events,Total events,43,97,95,76,110,102,...,237,257,271.0,300.0,301.0,302.0,302.0,306.0,339.0,329.0
72,Total disciplines,Total disciplines,Total disciplines,Total disciplines,10,22,18,13,25,18,...,31,34,37.0,40.0,40.0,42.0,40.0,42.0,50.0,48.0


In [11]:
import pandas as pd
import json

df=programs

# 处理缺失值：过滤掉 Discipline 为 NaN 的行
df_clean = df.dropna(subset=['Discipline'])

# 按 Sport 分组并提取唯一的 Discipline 列表
sport_discipline = (
    df_clean.groupby('Sport')['Discipline']
    .unique()                       # 去重
    .apply(list)                    # 转为列表
    .to_dict()                      # 转换为字典
)

# 处理原始数据中可能存在的 Sport 没有对应 Discipline 的情况
all_sports = df['Sport'].unique()
for sport in all_sports:
    if sport not in sport_discipline:
        sport_discipline[sport] = []  # 没有 Discipline 的 Sport 设为空列表

# 保存为 JSON 文件
with open('sport_discipline.json', 'w', encoding='utf-8') as f:
    json.dump(sport_discipline, f,
             ensure_ascii=False,  # 支持非英文字符
             indent=2)            # 美化格式

print("结果已保存至 sport_discipline.json")

结果已保存至 sport_discipline.json
